<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/Tokenizers_Foundation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Original word
text = "hello"

vocab = set()
for item in text:
    vocab.add(item)

vocab

{'e', 'h', 'l', 'o'}

### **Word-based Tokenizers**

In [ ]:
tokenized_text = "I am learning ProtonX".split()
print(tokenized_text)

['I', 'am', 'learning', 'ProtonX']


### **Sub-words Tokenizers**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
ids = tokenizer.encode("I am learning ProtonX")

In [ ]:
ids

[128000, 40, 1097, 6975, 1322, 783, 55]

In [ ]:
tokens = []
for id in ids:
    tokens.append(tokenizer.decode(id))

In [ ]:
tokens

['<|begin_of_text|>', 'I', ' am', ' learning', ' Pro', 'ton', 'X']

### Một số thuộc tính quan trọng

#### Từ điển

In [ ]:
# Get the vocabulary size
vocab_size = len(tokenizer)
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 128256


#### Thêm token đặc biệt

In [ ]:
tokenizer.add_special_tokens({
    "pad_token": "<PAD>",
})

0

In [ ]:
len(tokenizer)

128257

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>',
 'eos_token': '<|end_of_text|>',
 'pad_token': '<PAD>'}

#### Chỉnh lại embedding của model khi thay từ điển

Khi thêm token mới vào tokenizer, cần mở rộng lớp nhúng (embeddings) của mô hình để khớp với kích thước từ điển mới. Điều này đảm bảo mô hình xử lý đúng các token mới. Bạn có thể làm điều này bằng cách gọi phương thức `resize_token_embeddings` trên mô hình và truyền kích thước từ vựng đã cập nhật.

In [ ]:
from transformers import AutoModelForCausalLM

# Load the model
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

# Resize the model embeddings
model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 2048)

#### Lưu một tokenizer

In [ ]:
tokenizer.save_pretrained('./updated_tokenizer')

('./updated_tokenizer/tokenizer_config.json',
 './updated_tokenizer/special_tokens_map.json',
 './updated_tokenizer/tokenizer.json')

#### Load một tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./updated_tokenizer')

### Đào tạo một tokenizer

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
!gdown 1WTjIveEsM7XpN28xm6F1qgX57QxaoFI_

Downloading...
From: https://drive.google.com/uc?id=1WTjIveEsM7XpN28xm6F1qgX57QxaoFI_
To: /content/train_set.csv
100% 524k/524k [00:00<00:00, 109MB/s]


In [ ]:
import pandas as pd

# Đọc tệp CSV huấn luyện vào một DataFrame
train_df = pd.read_csv('/content/train_set.csv')

# Hiển thị vài hàng đầu tiên của DataFrame huấn luyện
print("Training Set:")
train_df.head()


Training Set:


,_id,label,title,label_numeric
0,66b5aabf8a38820e82e0b6ce,Xu hướng,"100+ STT Né thính, Cap né thính hài hước, NÉT ...",7
1,66b5a9838a38820e82e0b64d,Xu hướng,"Top 111+ stt cuộc sống an nhiên, bình dị tự tạ...",7
2,66b5cb358a38820e82e0c408,Xu hướng,"Top hạt giống hoa dễ trồng, nở quanh năm cho n...",7
3,66b5c7548a38820e82e0c271,Dinh dưỡng,Chi tiết 3 cách nấu rau bò khai đơn giản mà th...,1
4,66b5c7a78a38820e82e0c294,Nhà,Top 10 quạt cây hơi nước được ưa chuộng nhất h...,4


In [ ]:
example = train_df['title'][0]
tokens = old_tokenizer.tokenize("Trăm")
tokens

['Tr', 'Ä', 'ĥ', 'm']

In [ ]:
import pandas as pd

def get_training_corpus():
    return (
        train_df["title"][i : i + 1000].tolist()
        for i in range(0, len(train_df), 1000)
    )



In [ ]:
# Validate training corpus
training_corpus = get_training_corpus()
sample_batch = next(training_corpus, [])


if len(sample_batch) == 0:
    raise ValueError("Training corpus is empty. Ensure 'title' column has valid data.")

# Train new tokenizer
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, vocab_size=52000)

In [ ]:
# Test tokenizer with Vietnamese text
test_text = "Trăm"
tokenized_text = tokenizer.tokenize(test_text)

# Display tokenized output
print("Original Text:", test_text)
print("Tokenized Text:", tokenized_text)
print("Token Details:")
for i, token in enumerate(tokenized_text):
    print(f"  Token {i+1}: {token} -> {tokenizer.convert_tokens_to_string([token])}")


Original Text: Trăm
Tokenized Text: ['Tr', 'Äĥm']
Token Details:
  Token 1: Tr -> Tr
  Token 2: Äĥm -> ăm


### Lập trình BPE

In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/PhoGPT-4B")

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Hàm đếm các từ

In [ ]:
from collections import defaultdict

# Tạo một từ điển mặc định với giá trị khởi tạo là số nguyên 0
# Mục đích: Đếm tần suất xuất hiện của từng từ trong văn bản
word_freqs = defaultdict(int)

# Duyệt qua từng văn bản trong tập dữ liệu (corpus)
for text in corpus:
    # Tiền xử lý văn bản: Tokenizer chia văn bản thành các từ kèm vị trí (offsets)
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)

    # Tách chỉ lấy các từ từ kết quả tiền xử lý, bỏ qua phần offset
    new_words = [word for word, offset in words_with_offsets]

    # Duyệt qua từng từ và tăng giá trị đếm trong từ điển
    for word in new_words:
        word_freqs[word] += 1

# In ra từ điển chứa tần suất của từng từ
print(word_freqs)


defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


Từ điển gốc bao gồm ký tự

In [ ]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


Thêm token đặc biệt

In [ ]:
vocab = ["<|endoftext|>"] + alphabet.copy()

Bây giờ chúng ta cần tách từng từ thành các ký tự riêng lẻ để có thể bắt đầu quá trình huấn luyện:

In [ ]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [ ]:
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n'

 Hàm tính tần suất của từng cặp ký tự

In [ ]:
from collections import defaultdict

# Hàm tính tần suất xuất hiện của từng cặp ký tự trong các từ đã được tách
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)  # Khởi tạo một dictionary để lưu tần suất của các cặp ký tự, mặc định giá trị là 0

    # Duyệt qua từng từ và tần suất của nó trong tập dữ liệu
    for word, freq in word_freqs.items():
        split = splits[word]  # Lấy phiên bản đã tách của từ hiện tại

        # Nếu từ chỉ có một ký tự sau khi tách, bỏ qua
        if len(split) == 1:
            continue

        # Duyệt qua từng cặp ký tự liên tiếp trong từ
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])  # Tạo cặp ký tự liên tiếp
            pair_freqs[pair] += freq  # Cộng tần suất của từ vào cặp ký tự này

    return pair_freqs  # Trả về dictionary chứa tần suất của các cặp ký tự


In [ ]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


Tìm cặp xuất hiện nhiều nhất

In [ ]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


Hợp cặp này và thêm vào từ điển

In [ ]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [ ]:
# Hàm này thực hiện việc gộp một cặp ký tự (a, b) thành một ký tự duy nhất trong các từ đã được tách
def merge_pair(a, b, splits):
    # Duyệt qua từng từ trong tập dữ liệu
    for word in word_freqs:
        split = splits[word]  # Lấy danh sách ký tự đã tách của từ hiện tại

        # Nếu từ chỉ có một ký tự, bỏ qua vì không thể tạo cặp
        if len(split) == 1:
            continue

        i = 0  # Khởi tạo biến đếm i để duyệt qua các ký tự trong từ

        # Vòng lặp duyệt qua từng cặp ký tự trong từ
        # ['Ġ', 't', 'o']
        while i < len(split) - 1:
            # Kiểm tra nếu cặp ký tự hiện tại khớp với (a, b)
            if split[i] == a and split[i + 1] == b:
                # Gộp cặp ký tự (a, b) thành một ký tự duy nhất 'a+b'
                split = split[:i] + [a + b] + split[i + 2 :]
                # ['Ġt', 'o']
            else:
                i += 1  # Nếu không khớp, tăng i để kiểm tra cặp tiếp theo

        # Cập nhật lại từ đã được gộp trong splits
        splits[word] = split

    return splits  # Trả về splits đã được cập nhật sau khi gộp cặp ký tự


In [ ]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġto"])

['Ġt', 'o']


In [ ]:
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġt', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġt',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġt', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġt', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n', 'd', 'e', 'r', '

Lặp lại quá trình cho đến khi học được tất cả các phép gộp mà chúng ta mong muốn. Hãy đặt mục tiêu cho kích thước từ vựng là 50:

In [ ]:
# Đặt kích thước từ vựng mục tiêu là 50
vocab_size = 50

# Vòng lặp tiếp tục cho đến khi kích thước từ vựng đạt đến kích thước mục tiêu
while len(vocab) < vocab_size:
    # Bước 1: Tính tần suất của tất cả các cặp ký tự trong các từ đã được tách
    pair_freqs = compute_pair_freqs(splits)

    best_pair = ""  # Khởi tạo cặp ký tự tốt nhất
    max_freq = None  # Khởi tạo tần suất cao nhất là None

    # Bước 2: Tìm cặp ký tự xuất hiện nhiều nhất
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair  # Cập nhật cặp ký tự có tần suất cao nhất
            max_freq = freq  # Cập nhật tần suất cao nhất

    # Bước 3: Gộp cặp ký tự tốt nhất vào các từ đã tách
    splits = merge_pair(*best_pair, splits)

    # Bước 4: Lưu lại phép gộp vào từ điển 'merges' để theo dõi
    merges[best_pair] = best_pair[0] + best_pair[1]

    # Bước 5: Thêm cặp ký tự đã gộp vào danh sách từ vựng
    vocab.append(best_pair[0] + best_pair[1])


**Ký tự Ġ** đại diện cho khoảng trắng

In [ ]:
print(merges)


{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('i', 'n'): 'in', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni'}


In [ ]:
print(vocab)


['<|endoftext|>', ',', '.', 'C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġab', 'Ġtokeni']


Tokenize văn bản mới

In [ ]:
# Hàm tokenize chuyển đổi văn bản thành các token theo các quy tắc đã học được từ quá trình huấn luyện
def tokenize(text):
    # Bước 1: Tiền xử lý văn bản để tách thành các từ
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    # Lấy danh sách các từ từ kết quả tiền xử lý (bỏ qua phần offset)

    # Bước 2: Tách từng từ thành danh sách các ký tự riêng lẻ
    splits = [[l for l in word] for word in pre_tokenized_text]

    print('---splits', splits)

    # Bước 3: Áp dụng các phép gộp đã học được để hợp nhất các cặp ký tự
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):  # Duyệt qua từng từ đã tách thành ký tự
            i = 0  # Khởi tạo chỉ số i để duyệt qua từng ký tự trong từ

            # Duyệt qua từng cặp ký tự trong từ
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    # Nếu cặp ký tự trùng khớp với cặp cần gộp, thay thế bằng ký tự đã gộp
                    split = split[:i] + [merge] + split[i + 2 :]
                    print("-merged", split)
                else:
                    i += 1  # Nếu không khớp, tiếp tục kiểm tra cặp tiếp theo

            splits[idx] = split  # Cập nhật lại từ đã được xử lý


    # Bước 4: Gộp tất cả các danh sách ký tự thành một danh sách duy nhất (tokenized text)
    return sum(splits, [])


In [ ]:
tokenize("This is not a token.")

---splits [['T', 'h', 'i', 's'], ['Ġ', 'i', 's'], ['Ġ', 'n', 'o', 't'], ['Ġ', 'a'], ['Ġ', 't', 'o', 'k', 'e', 'n'], ['.']]
-merged ['Ġt', 'o', 'k', 'e', 'n']
-merged ['T', 'h', 'is']
-merged ['Ġ', 'is']
-merged ['Ġa']
-merged ['Ġto', 'k', 'e', 'n']
-merged ['Ġto', 'k', 'en']
-merged ['Th', 'is']
-merged ['This']
-merged ['Ġtok', 'en']
-merged ['Ġtoken']
-merged ['Ġis']


['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

In [ ]:
def detokenize(tokens):
    """
    Chuyển danh sách token thành chuỗi văn bản gốc.
    """
    text = ""
    for token in tokens:
        if token.startswith("Ġ"):
            text += " " + token[1:]  # Thêm khoảng trắng trước token nếu bắt đầu bằng 'Ġ'
        else:
            text += token  # Thêm token trực tiếp nếu không có 'Ġ'
    return text.strip()  # Loại bỏ khoảng trắng thừa ở đầu và cuối chuỗi

# Ví dụ sử dụng
tokens = ['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']
text = detokenize(tokens)
print(text)


This is not a token.


### Đào tạo WordPiece

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [ ]:
import pandas as pd

def get_training_corpus():
    return (
        train_df["title"][i : i + 1000].tolist()
        for i in range(0, len(train_df), 1000)
    )

Bước đầu tiên của quá trình tokenization là chuẩn hóa (normalization), vì vậy hãy bắt đầu với bước này. Vì BERT được sử dụng rộng rãi, nên có một `BertNormalizer` với các tùy chọn cổ điển mà chúng ta có thể thiết lập cho BERT:

- **`lowercase`** (chuyển thành chữ thường) và **`strip_accents`** (loại bỏ dấu), cả hai đều dễ hiểu.  
- **`clean_text`** (làm sạch văn bản) để loại bỏ tất cả các ký tự điều khiển và thay thế các khoảng trắng lặp lại bằng một khoảng trắng duy nhất.  
- **`handle_chinese_chars`** (xử lý ký tự Trung Quốc), thêm khoảng trắng xung quanh các ký tự Trung Quốc.  

Để tái tạo tokenizer `bert-base-uncased`, chúng ta chỉ cần thiết lập bộ chuẩn hóa này.

### **Ví dụ về các ký tự điều khiển**

Dưới đây là một số ký tự điều khiển phổ biến:

- **Newline (`\n`)** – Di chuyển con trỏ xuống dòng tiếp theo.  
- **Carriage Return (`\r`)** – Di chuyển con trỏ về đầu dòng.  
- **Tab (`\t`)** – Chèn một khoảng trắng dạng tab.  
- **Backspace (`\b`)** – Di chuyển con trỏ lùi lại một ký tự.  
- **Form Feed (`\f`)** – Di chuyển con trỏ đến đầu trang tiếp theo.  
- **Escape (`\x1b`)** – Được sử dụng để bắt đầu một chuỗi điều khiển.  
- **Bell (`\a`)** – Phát ra âm thanh trên một số hệ thống.  

In [ ]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

Tùy biến các hàm clean

### 📚 **Giải Thích Chi Tiết Cho Normalizer Sequence**

Dòng mã bạn cung cấp thiết lập một **chuỗi (Sequence)** các bước chuẩn hóa văn bản cho tokenizer. Dưới đây là giải thích chi tiết cho từng bước:

---

### ⚙️ **1. normalizers.NFD()**

- **Chức năng:** Thực hiện **chuẩn hóa NFD (Normalization Form Decomposition)**.  
- **Cách hoạt động:** Phân tách các ký tự có dấu thành ký tự cơ bản và dấu tách rời.  
- **Ví dụ:**  
   - `ắ` → `a` + `̆` + `́`  
   - `ỗ` → `o` + `̂` + `̃`  

**✅ Mục đích:** Giúp xử lý văn bản ở cấp độ ký tự, đặc biệt khi cần loại bỏ dấu hoặc thực hiện các thao tác trên từng thành phần.

---

### ⚙️ **2. normalizers.Lowercase()**

- **Chức năng:** Chuyển tất cả các ký tự thành **chữ thường**.  
- **Ví dụ:**  
   - `VIỆT NAM` → `việt nam`  
   - `Hà Nội` → `hà nội`  

**✅ Mục đích:** Đảm bảo tính nhất quán, giảm nhầm lẫn khi so sánh văn bản.

---

### ⚙️ **3. normalizers.StripAccents()**

- **Chức năng:** **Loại bỏ tất cả dấu (diacritics)** khỏi ký tự.  
- **Yêu cầu:** Thường được kết hợp sau khi chuẩn hóa **NFD** (để dấu tách rời dễ dàng loại bỏ).  
- **Ví dụ:**  
   - `ắ` → `a`  
   - `ỗ` → `o`  

**✅ Mục đích:** Dùng cho các bài toán không quan tâm đến dấu, như tìm kiếm không phân biệt dấu.

---

### 🛠️ **Chuỗi Hoạt Động Của Normalizer**

1. **NFD:** Phân tách ký tự và dấu.  
2. **Lowercase:** Chuyển tất cả ký tự thành chữ thường.  
3. **StripAccents:** Loại bỏ dấu khỏi các ký tự.

---

### 📝 **Ví Dụ Cụ Thể**

**Đầu vào:**  
```text
"Hà Nội Đẹp Quá!"
```

**Quy trình qua các bước:**

1. **NFD:**  
   - `Hà Nội Đẹp Quá!` → `Ha` + `̀` + ` No` + `̣` + `i De` + `̣` + `p Qua` + `́` + `!`  

2. **Lowercase:**  
   - `ha` + `̀` + ` no` + `̣` + `i de` + `̣` + `p qua` + `́` + `!`  

3. **StripAccents:**  
   - `ha noi dep qua!`

**Kết quả cuối cùng:**  
```
"ha noi dep qua!"
```

---

### ✅ **Khi Nào Sử Dụng Chuỗi Này?**

- **Tìm kiếm văn bản:** Bỏ qua dấu và phân biệt chữ hoa/chữ thường.  
- **Phân tích văn bản:** Đơn giản hóa văn bản cho các mô hình NLP.  
- **So khớp từ khóa:** Giảm sai lệch do sự khác biệt về dấu hoặc chữ hoa.

---

Nếu bạn có thêm câu hỏi hoặc muốn mở rộng ví dụ, hãy cho mình biết nhé! 🚀

In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [ ]:
print(tokenizer.normalizer.normalize_str("Tôi đi học"))

toi đi hoc


### 📚 **Giải Thích `tokenizer.normalizer` với `normalizers.NFC()`**

Dòng mã trên thiết lập một bước chuẩn hóa cho tokenizer sử dụng **NFC (Normalization Form Composition)**. Dưới đây là chi tiết về cách hoạt động và lý do sử dụng NFC.

---

## ⚙️ **1. normalizers.NFC()**

- **Chức năng:** Thực hiện **chuẩn hóa NFC (Normalization Form Composition)**.  
- **Cách hoạt động:** Kết hợp các ký tự cơ bản và dấu (diacritics) thành một ký tự duy nhất nếu có thể.  
- **Ví dụ:**  
   - `a` + `́` → `á`  
   - `o` + `̂` + `̃` → `ỗ`  

### ✅ **Mục đích:**  
- Đảm bảo tính nhất quán trong cách biểu diễn ký tự Unicode.  
- Phù hợp cho lưu trữ, hiển thị, và xử lý văn bản trong các hệ thống không hỗ trợ NFD.  
- Giảm sai lệch khi so sánh chuỗi văn bản.


In [ ]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFC(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [ ]:
print(tokenizer.normalizer.normalize_str("Tôi đi học"))

tôi đi học


Bộ đào tạo

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [ ]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
encoding = tokenizer.encode("Tôi đi học")
print(encoding.tokens)

['tô', '##i đ', '##i h', '##ọc']


In [ ]:
print(encoding.ids)


[7041, 12035, 1037, 548]


In [ ]:
tokenizer.decode(encoding.ids)

'tô ##i đ ##i h ##ọc'

Đọc thêm chi tiết: https://huggingface.co/learn/nlp-course/en/chapter6/8?fw=pt

In [ ]:
# Tôi đi xe máy

# Tôi đi xe_máy